In [10]:
'''Restore a character-level sequence to sequence model from disk and use it
to generate predictions.
This script loads the s2s.h5 model saved by lstm_seq2seq.py and generates
sequences from it.  It assumes that no changes have been made (for example:
latent_dim is unchanged, and the input data and model architecture are unchanged).
See lstm_seq2seq.py for more details on the model architecture and how
it is trained.
'''

from __future__ import print_function

from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input
import numpy as np
from utils import *
from sklearn.preprocessing import StandardScaler

In [11]:
""" Inference model

# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state and the last known FG value.
# 3) The output of the decoder will be the next target FG. Repeat with the 
current target FG and current states""" 

def load_inference_model(path_name, print_summary = False):
    
    latent_dim = 256  # Latent dimensionality of the encoding space.
    
    # Restore the model and construct the encoder and decoder.
    model = load_model(path_name)

    #Create encoder model
    encoder_inputs = model.input[0]   # input_encoder
    encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output # encoder_layer
    encoder_states = [state_h_enc, state_c_enc]

    encoder_model = Model(encoder_inputs, encoder_states)
    
    if print_summary:
        encoder_model.summary()

    # Create decoder model
    decoder_inputs = model.input[1]   # input_decoder
    decoder_state_input_h = Input(shape=(latent_dim,), name='decoder_input_h')
    decoder_state_input_c = Input(shape=(latent_dim,), name='decoder_input_c')
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

    decoder_lstm = model.layers[3] # decoder_layer
    decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
        decoder_inputs, initial_state=decoder_states_inputs)

    decoder_states = [state_h_dec, state_c_dec]

    decoder_dense = model.layers[4] # Dense layer

    decoder_outputs = decoder_dense(decoder_outputs)

    decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)
    
    if print_summary:
        decoder_model.summary()
    
    return encoder_model, decoder_model

In [12]:
def predict_FGs(encoder_model, decoder_model, input_data, last_known_FG, maxi = 10):
    
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_data)
        
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, 1))
    
    # Populate the first FG of target sequence with the last known FG.
    target_seq[0, 0, 0] = last_known_FG
    
    predicted_FGs = np.empty((maxi))
    
    i= 0
    
    while i < maxi:
        
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        predicted_FG = output_tokens.ravel()[0]
        
        # Sample a token
        predicted_FGs[i] = predicted_FG

        # Update the target sequence.
        target_seq = np.zeros((1, 1, 1))
        target_seq[0, 0, 0] = predicted_FG

        # Update states
        states_value = [h, c]
        
        i+=1

    return predicted_FGs

In [17]:
# Example of inference

imputation_method= "mean-average_before_after_values"
output_path= "model-results/"

dataset = pd.read_csv(
        "datasets/datos-demograficos-visitas-tratamientos-missing-imputation-{method}-allnumerics.csv".format(
            method=imputation_method))

dataset_original = pd.read_csv(
        "datasets/datos-demograficos-visitas-tratamientos-missing-imputation-{method}.csv".format(
            method=imputation_method))

num_features = dataset.shape[1] - 1  # Id is not considered

scaler = StandardScaler()
scaler.fit(dataset_original["Filtrado_glomerular"].values.reshape(-1,1))

encoder_input_data, decoder_input_data, decoder_target_data, buckets = transform_dataset_different_timesteps(
        dataset)

# Loading the model
encoder_model, decoder_model = load_inference_model(output_path + "{method}-Fold-0.best.hdf5".format(method= imputation_method))

In [28]:
i = 11

visitas = encoder_input_data[i]
dec_x = decoder_input_data[i]

timesteps_visitas = int(len(visitas) / num_features)
last_knownFG = dec_x[0]

visitas = np.array(visitas).reshape((1, timesteps_visitas, num_features))

predictions = predict_FGs(encoder_model, decoder_model, visitas, last_knownFG, 6)

scaler.inverse_transform(predictions)

array([13.23412043, 10.92374239, 10.9780811 , 10.92499236, 10.79870438,
       10.58165195])

In [29]:
visitas.shape

(1, 2, 70)